In [ ]:
# Imports minimaux nécessaires
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import validate_data, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn.naive_bayes import GaussianNB  # Fallback simple

In [ ]:
class BayesianClassifier(ClassifierMixin, BaseEstimator):
    """Classifieur Bayésien minimal pour scikit-learn"""
    
    def __init__(self, model=None):
        self.model = model
    
    def fit(self, X, y):
        # Validation sklearn standard
        X, y = validate_data(self, X, y)
        self.classes_ = unique_labels(y)
        
        # Si pas de modèle fourni, utiliser Naive Bayes simple
        if self.model is None:
            self.model = GaussianNB()
        
        # Entraîner le modèle
        self.model.fit(X, y)
        return self
    
    def predict(self, X):
        check_is_fitted(self)
        X = validate_data(self, X, reset=False)
        return self.model.predict(X)
    
    def predict_proba(self, X):
        check_is_fitted(self)
        X = validate_data(self, X, reset=False)
        return self.model.predict_proba(X)

In [ ]:
# Test simple du classifieur
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Données de test
X, y = make_classification(n_samples=100, n_features=4, n_classes=2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Test du classifieur
classifier = BayesianClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(f"Précision: {accuracy_score(y_test, y_pred):.3f}")
print("✅ Classifieur minimal fonctionnel !")

## Question 1: Classifieur Bayésien Minimal

**Version simplifiée au maximum** qui respecte le template sklearn:

### Fonctionnalités:
- ✅ **API sklearn complète**: `fit()`, `predict()`, `predict_proba()`
- ✅ **Modèle flexible**: Accepte un modèle pré-défini ou utilise Naive Bayes par défaut
- ✅ **Validation des données**: Compatible avec l'écosystème sklearn
- ✅ **Code minimal**: ~20 lignes seulement

### Utilisation:
```python
# Utilisation basique (Naive Bayes automatique)
classifier = BayesianClassifier()
classifier.fit(X_train, y_train)

# Avec un modèle personnalisé
from sklearn.naive_bayes import MultinomialNB
classifier = BayesianClassifier(model=MultinomialNB())
```

Cette version minimaliste peut être étendue selon les besoins spécifiques.

### graphical models